In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
import torch.optim as optim
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import os, glob

In [27]:
!unzip response.zip -d data

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: data/04 - AIXI Shape/train/26319.png  
  inflating: data/04 - AIXI Shape/train/26320.png  
  inflating: data/04 - AIXI Shape/train/26321.png  
  inflating: data/04 - AIXI Shape/train/26322.png  
  inflating: data/04 - AIXI Shape/train/26323.png  
  inflating: data/04 - AIXI Shape/train/26324.png  
  inflating: data/04 - AIXI Shape/train/26325.png  
  inflating: data/04 - AIXI Shape/train/26326.png  
  inflating: data/04 - AIXI Shape/train/26327.png  
  inflating: data/04 - AIXI Shape/train/26328.png  
  inflating: data/04 - AIXI Shape/train/26329.png  
  inflating: data/04 - AIXI Shape/train/26330.png  
  inflating: data/04 - AIXI Shape/train/26331.png  
  inflating: data/04 - AIXI Shape/train/26332.png  
  inflating: data/04 - AIXI Shape/train/26333.png  
  inflating: data/04 - AIXI Shape/train/26334.png  
  inflating: data/04 - AIXI Shape/train/26335.png  
  inflating: data/04 - AIXI Shape/train/26336.png  
  i

In [28]:
N_IMAGES_TRAIN = 20_000
TRAIN_BATCH_SIZE = 128
TEST_BATCH_SIZE = 100

In [29]:
# First we define a custom dataset class to be able to create the DataLoader properly

class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, labels_file, img_dir, separator=',', img_type="png",transform=None, max_images=None):
        # As labels, we select the 'c' column of the dataset, which contains the number
        # of circles of the image
        aux = pd.read_csv(labels_file,sep=separator)
        aux.drop(columns=aux.columns[0], axis=1,inplace=True)
        aux.drop(columns=aux.columns[1:], axis=1, inplace=True)
        self.img_labels = aux.to_numpy().squeeze()
        if max_images is not None:
            self.img_labels = self.img_labels[:max_images]
        self.img_dir = img_dir
        self.transform = transform
        self.img_type = img_type

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        # To get the images, we transform the index to a 5 long string (so 28 -> 00028)
        # and we use that to read the image.
        img_name = str(idx).zfill(5)+'.'+self.img_type
        img_path = os.path.join(self.img_dir, img_name)
        image = cv2.imread(img_path,  cv2.IMREAD_GRAYSCALE)
        label = self.img_labels[idx,]
        if self.transform:
            image = self.transform(image)
            image = image.float()
        label = torch.tensor(label, dtype=torch.float32)
        return (image, label)

In [30]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((128, 128))    
    ])

train_dataset = CustomImageDataset(labels_file='data/train/dades.csv', img_dir='data/train', separator=';', max_images=N_IMAGES_TRAIN, transform=transform)
test_dataset = CustomImageDataset(labels_file='data/val/dades.csv', img_dir='data/val', separator=';', transform=transform)

train_loader = DataLoader(train_dataset, TRAIN_BATCH_SIZE)
test_loader = DataLoader(test_dataset, TEST_BATCH_SIZE)

In [31]:
# Now using the AlexNet
AlexNet_model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=False)

# Model description
AlexNet_model.eval()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [32]:
# Updating the first convolutional layer to work with 1 channel input images
AlexNet_model.features[0] = nn.Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))

# Updating the second classifier
AlexNet_model.classifier[4] = nn.Linear(4096, 1024)

# Updating the third and the last "classifier" that is the output layer of the network. Make sure to have only 1 output node if we are going to transform our model into a regression one.
AlexNet_model.classifier[6] = nn.Linear(1024, 1)

# Modified model description
AlexNet_model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [33]:
def train(model, device, train_loader, optimizer, epoch, log_interval=100, verbose=True):
    
    model.train()

    loss_v = 0

    for batch_idx, (data, target) in enumerate(train_loader):
    
        data, target = data.to(device), target.to(device)
        output = model(data)

        loss = loss_func(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0 and verbose:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Average: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item(), loss.item() / len(data)))
        loss_v += loss.item()

    loss_v /= len(train_loader.dataset)
    print('\nTrain set: Average loss: {:.4f}\n'.format(loss_v))
 
    return loss_v


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_func(output, target)
            output = torch.round(output)
            correct += output.eq(target.view_as(output)).sum().item()
 
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    return test_loss

In [35]:
use_cuda = True
torch.manual_seed(33)

if use_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

epochs = 20
lr = 0.01

model = AlexNet_model.to(device)

pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Parameters ", pytorch_total_params)
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_func = torch.nn.MSELoss()

# Guardam el valor de pèrdua promig de cada iteració (època)
train_l = np.zeros((epochs))
test_l = np.zeros((epochs))

# Bucle d'entrenament
for epoch in range(0, epochs):
    train_l[epoch] = train(model, device, train_loader, optimizer, epoch)
    test_l[epoch]  = test(model, device, test_loader)

Parameters  44403393
Train Epoch: 0 [0/20000 (0%)]	Loss: 0.658824, Average: 0.005147
Train Epoch: 0 [12800/20000 (64%)]	Loss: 1.425521, Average: 0.011137

Train set: Average loss: 0.0251


Test set: Average loss: 0.0067, Accuracy: 694/2000 (35%)

Train Epoch: 1 [0/20000 (0%)]	Loss: 0.671203, Average: 0.005244
Train Epoch: 1 [12800/20000 (64%)]	Loss: 0.676710, Average: 0.005287

Train set: Average loss: 0.0054


Test set: Average loss: 0.0067, Accuracy: 694/2000 (35%)

Train Epoch: 2 [0/20000 (0%)]	Loss: 0.661739, Average: 0.005170


KeyboardInterrupt: ignored